## Data preparation for cyclegan
This notebook will allow us to move the right files to prepare cyclegan from one illuminant to another.

In [1]:
import os
import pandas as pd
import numpy
import sys
import shutil
from pathlib import Path

In [2]:
pd.read_csv('/scratch/barthas/data/train/scene_abandonned_city_54/2500/E/dataset_2500_E.csv').columns


Index(['Unnamed: 0', 'scene', 'illuminant', 'scene_center_x', 'scene_center_y',
       'scene_center_z', 'rendered_image', 'camera_location_x',
       'camera_location_y', 'camera_location_z', 'NW', 'N', 'NE', 'E', 'SE',
       'S', 'SW', 'W', 'white_balance', 'exposure_compensation',
       'pointlight_intensity'],
      dtype='object')

In [16]:
# Desired illuminants {A, B}
color_temperatures = {'A': 4500, 'B': 4500}
locations = {'A': 'SW','B': 'SE'}

destination_dataset_path = '/scratch/barthas/datasets/SW_SE/'
Path(destination_dataset_path).mkdir(parents=True, exist_ok=True)
Path(os.path.join(destination_dataset_path, 'train')).mkdir(parents=True, exist_ok=True)
Path(os.path.join(destination_dataset_path, 'test')).mkdir(parents=True, exist_ok=True)
Path(os.path.join(destination_dataset_path, 'validate')).mkdir(parents=True, exist_ok=True)
Path(os.path.join(destination_dataset_path, 'train', 'A')).mkdir(parents=True, exist_ok=True)
Path(os.path.join(destination_dataset_path, 'train', 'B')).mkdir(parents=True, exist_ok=True)
Path(os.path.join(destination_dataset_path, 'test', 'A')).mkdir(parents=True, exist_ok=True)
Path(os.path.join(destination_dataset_path, 'test', 'B')).mkdir(parents=True, exist_ok=True)
Path(os.path.join(destination_dataset_path, 'validate', 'A')).mkdir(parents=True, exist_ok=True)
Path(os.path.join(destination_dataset_path, 'validate', 'B')).mkdir(parents=True, exist_ok=True)

train_path = '/scratch/barthas/data/train/'
test_path = '/scratch/barthas/data/test/'
validate_path = '/scratch/barthas/data/validate/'

train_scenes = os.listdir(train_path)
test_scenes = os.listdir(test_path)
validate_scenes = os.listdir(validate_path)


col_names = ['Unnamed: 0', 'scene', 'illuminant', 'scene_center_x', 'scene_center_y',\
             'scene_center_z', 'rendered_image', 'camera_location_x',\
             'camera_location_y', 'camera_location_z', 'NW', 'N', 'NE', 'E', 'SE',\
             'S', 'SW', 'W', 'white_balance', 'exposure_compensation','pointlight_intensity']


def generate_dataset_both(illuminant = 'A', step = 'train', columns = col_names):
    dataset  = pd.DataFrame(columns = col_names)
    for index, scene in enumerate(os.listdir(os.path.join('/scratch/barthas/','data', step))):
        data_path = os.path.join('/scratch/barthas/', 'data', step, scene, str(color_temperatures[illuminant]), locations[illuminant])
        Path(data_path).mkdir(parents=True, exist_ok=True)
        dataset_name = 'dataset_' + str(color_temperatures[illuminant]) + '_' + locations[illuminant] + '.csv'
        data = pd.read_csv(os.path.join(data_path, dataset_name))
        # Rename images and copy them to avoid duplicates
        for image_name in data['rendered_image']:
            old_image_name = os.path.join(data_path, image_name)        
            # Separate base from extension
            base, extension = os.path.splitext(image_name)        
            new_image_name = os.path.join(destination_dataset_path, step, illuminant, str(index) + '_' + base + extension)
            shutil.copy(old_image_name, new_image_name)
        data['rendered_image'] = data['rendered_image'].map(lambda image_name:\
                                                            str(index) + '_' + image_name)
        dataset = dataset.append(data)
    #dataset.to_csv(os.path.join(destination_dataset_path, 'train', 'A', 'dataset_A.csv'))
    return dataset

#dataset_A = generate_dataset_both(illuminant = 'A', step = 'train', columns = col_names)
#dataset_A.head()

locations_list = ['SW', 'S', 'SE', 'E', 'NE', 'N', 'NW', 'W']

def generate_dataset_color_temperature(illuminant = 'A', step ='train', columns = col_names):
    dataset  = pd.DataFrame(columns = col_names)
    for index, scene in enumerate(os.listdir(os.path.join('/scratch/barthas/','data', step))):
        for location in locations_list:
            data_path = os.path.join('/scratch/barthas/','data', step, scene, str(color_temperatures[illuminant]), location)
            dataset_name = 'dataset_' + str(color_temperatures[illuminant]) + '_' + location + '.csv'
            data = pd.read_csv(os.path.join(data_path, dataset_name))
            # Rename images and copy them to avoid duplicates
            for image_name in data['rendered_image']:
                old_image_name = os.path.join(data_path, image_name)        
                # Separate base from extension
                base, extension = os.path.splitext(image_name)        
                new_image_name = os.path.join(destination_dataset_path, step, illuminant, str(index) + '_' + base + extension)
                shutil.copy(old_image_name, new_image_name)
            data['rendered_image'] = data['rendered_image'].map(lambda image_name:\
                                                                str(index) + '_' + image_name)
            dataset = dataset.append(data)
    #dataset.to_csv(os.path.join(destination_dataset_path, 'train', 'A', 'dataset_A.csv'))
    return dataset

In [4]:
dataset_A = generate_dataset_color_temperature(illuminant = 'A', step = 'train', columns = col_names)
dataset_A.head()
#dataset_A = generate_dataset_both(illuminant = 'A', step = 'train', columns = col_names)
#dataset_A.head()


,Unnamed: 0,scene,illuminant,scene_center_x,scene_center_y,scene_center_z,rendered_image,camera_location_x,camera_location_y,camera_location_z,...,N,NE,E,SE,S,SW,W,white_balance,exposure_compensation,pointlight_intensity
0,30,Landscape12020119222725,2500.0,15129.172852,10193.940430,263.128418,0_image30.png,15363.064453,10035.955078,386.719604,...,0,0,0,0,0,1,0,6500.0,0.0,331.325653Candelas
1,70,S_Cliff_Large_142020119222734,2500.0,13883.899414,8892.037109,102.170166,0_image70.png,14086.230469,8951.983398,174.300888,...,0,0,0,0,0,1,0,6500.0,0.0,239.806686Candelas
2,110,S_Cliff_Large_142020119222738,2500.0,13877.687500,8818.366211,112.674622,0_image110.png,14010.418945,8715.196289,150.788803,...,0,0,0,0,0,1,0,6500.0,0.0,185.359619Candelas
3,150,Signpost_OneDir32020119222745,2500.0,13502.585938,8283.014648,376.527130,0_image150.png,13822.921875,8197.805664,435.604462,...,0,0,0,0,0,1,0,6500.0,0.0,362.055756Candelas
4,190,WoodenFence_Small72020119222757,2500.0,13357.870117,7861.131836,222.214508,0_image190.png,13488.060547,7824.246582,267.690277,...,0,0,0,0,0,1,0,6500.0,0.0,153.50145Candelas


In [5]:
dataset_B = generate_dataset_color_temperature(illuminant = 'B', step = 'train', columns = col_names)
dataset_B.head()
#dataset_B = generate_dataset_both(illuminant = 'B', step = 'train', columns = col_names)
#dataset_B.head()

,Unnamed: 0,scene,illuminant,scene_center_x,scene_center_y,scene_center_z,rendered_image,camera_location_x,camera_location_y,camera_location_z,...,N,NE,E,SE,S,SW,W,white_balance,exposure_compensation,pointlight_intensity
0,34,Landscape12020119222725,6500.0,15129.172852,10193.940430,263.128418,0_image34.png,15363.064453,10035.955078,386.719604,...,0,0,0,0,0,1,0,6500.0,0.0,331.325653Candelas
1,74,S_Cliff_Large_142020119222734,6500.0,13883.899414,8892.037109,102.170166,0_image74.png,14086.230469,8951.983398,174.300888,...,0,0,0,0,0,1,0,6500.0,0.0,239.806686Candelas
2,114,S_Cliff_Large_142020119222738,6500.0,13877.687500,8818.366211,112.674622,0_image114.png,14010.418945,8715.196289,150.788803,...,0,0,0,0,0,1,0,6500.0,0.0,185.359619Candelas
3,154,Signpost_OneDir32020119222745,6500.0,13502.585938,8283.014648,376.527130,0_image154.png,13822.921875,8197.805664,435.604462,...,0,0,0,0,0,1,0,6500.0,0.0,362.055756Candelas
4,194,WoodenFence_Small72020119222757,6500.0,13357.870117,7861.131836,222.214508,0_image194.png,13488.060547,7824.246582,267.690277,...,0,0,0,0,0,1,0,6500.0,0.0,153.50145Candelas


In [17]:
dataset_C = generate_dataset_color_temperature(illuminant = 'A', step = 'test', columns = col_names)
dataset_C.head()
#dataset_C = generate_dataset_both(illuminant = 'A', step = 'test', columns = col_names)
#dataset_C.head()

,Unnamed: 0,scene,illuminant,scene_center_x,scene_center_y,scene_center_z,rendered_image,camera_location_x,camera_location_y,camera_location_z,...,N,NE,E,SE,S,SW,W,white_balance,exposure_compensation,pointlight_intensity
0,32,Case_A62019101820144,4500.0,-1522.750854,-44977.019531,-76.677216,0_image32.png,-1569.822021,-44711.058594,-20.700724,...,0,0,0,0,0,1,0,6500.0,0.0,296.608978Candelas
1,72,Doorway_Small_A1582019101820148,4500.0,-1443.810303,-45160.597656,-37.968674,0_image72.png,-1610.689819,-45260.164063,31.629517,...,0,0,0,0,0,1,0,6500.0,0.0,221.957352Candelas
2,112,Doorway_Small_A1582019101820152,4500.0,-1800.562500,-45014.726563,-64.201050,0_image112.png,-1667.670898,-45245.316406,31.566835,...,0,0,0,0,0,1,0,6500.0,0.0,304.147308Candelas
3,152,Doorway_Small_A104201910182023,4500.0,-1495.390869,-44148.777344,-149.615189,0_image152.png,-1448.622070,-44386.449219,-50.483757,...,0,0,0,0,0,1,0,6500.0,0.0,281.437805Candelas
4,192,WaterDrum24201910182027,4500.0,-2558.905273,-44293.867188,-23.557648,0_image192.png,-2127.442383,-44174.105469,-61.720314,...,0,0,0,0,0,1,0,6500.0,0.0,483.240906Candelas


In [18]:
dataset_D = generate_dataset_color_temperature(illuminant = 'B', step = 'test', columns = col_names)
dataset_D.head()
#dataset_D = generate_dataset_both(illuminant = 'B', step = 'test', columns = col_names)
#dataset_D.head()

,Unnamed: 0,scene,illuminant,scene_center_x,scene_center_y,scene_center_z,rendered_image,camera_location_x,camera_location_y,camera_location_z,...,N,NE,E,SE,S,SW,W,white_balance,exposure_compensation,pointlight_intensity
0,22,Case_A62019101820144,4500.0,-1522.750854,-44977.019531,-76.677216,0_image22.png,-1569.822021,-44711.058594,-20.700724,...,0,0,0,1,0,0,0,6500.0,0.0,296.608978Candelas
1,62,Doorway_Small_A1582019101820148,4500.0,-1443.810303,-45160.597656,-37.968674,0_image62.png,-1610.689819,-45260.164063,31.629517,...,0,0,0,1,0,0,0,6500.0,0.0,221.957352Candelas
2,102,Doorway_Small_A1582019101820152,4500.0,-1800.562500,-45014.726563,-64.201050,0_image102.png,-1667.670898,-45245.316406,31.566835,...,0,0,0,1,0,0,0,6500.0,0.0,304.147308Candelas
3,142,Doorway_Small_A104201910182023,4500.0,-1495.390869,-44148.777344,-149.615189,0_image142.png,-1448.622070,-44386.449219,-50.483757,...,0,0,0,1,0,0,0,6500.0,0.0,281.437805Candelas
4,182,WaterDrum24201910182027,4500.0,-2558.905273,-44293.867188,-23.557648,0_image182.png,-2127.442383,-44174.105469,-61.720314,...,0,0,0,1,0,0,0,6500.0,0.0,483.240906Candelas


In [ ]:
dataset_E = generate_dataset_color_temperature(illuminant = 'A', step = 'validate', columns = col_names)
dataset_E.head()
#dataset_E = generate_dataset_both(illuminant = 'A', step = 'validate', columns = col_names)
#dataset_E.head()

In [ ]:
dataset_F = generate_dataset_color_temperature(illuminant = 'B', step = 'validate', columns = col_names)
dataset_F.head()
#dataset_F = generate_dataset_both(illuminant = 'B', step = 'validate', columns = col_names)
#dataset_F.head()